In [181]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm

In [336]:
def average_angle_ignore_nan(degrees):
    """
    주어진 각도의 리스트에서 NaN 값을 무시하고 평균을 계산합니다.
    
    :param degrees: 각도의 리스트 (0-360도)
    :return: 평균 각도 (0-360도)
    """
    # NaN 값을 무시하고 유효한 각도만 선택
    valid_degrees = [deg for deg in degrees if not pd.isna(deg)]
    
    if not valid_degrees:
        return np.nan  # 유효한 각도가 없는 경우 NaN 반환
    
    # 유효한 각도를 라디안으로 변환
    radians = np.deg2rad(valid_degrees)
    
    # x, y 좌표 계산
    x_coords = np.cos(radians)
    y_coords = np.sin(radians)
    
    # x, y 좌표의 평균 계산
    x_mean = np.mean(x_coords)
    y_mean = np.mean(y_coords)
    
    # 평균 좌표를 각도로 변환
    mean_rad = np.arctan2(y_mean, x_mean)
    mean_deg = np.rad2deg(mean_rad)
    
    # 결과를 0-360도 사이의 값으로 변환
    mean_deg = mean_deg % 360
    
    return mean_deg

In [337]:
def horizontal_average(dataframe, datetime, column, threshold = 3, digits = 1):
    '''
    동일한 시간의 여러 지점 데이터를 확인한 후 해당 데이터들의 평균값을 반환하는 함수
    threshold 값을 입력하여 허용하는 NaN 값의 최대치를 설정할 수 있음
    '''  
    # 입력된 일시와 컬럼명에 해당하는 데이터 변수화
    data = dataframe.loc[datetime, column]
    
    # 임시로 nan값 입력
    mean = np.nan
    
    # data에 NaN값의 수가 threshold를 초과하는지 확인
    if data.isna().sum() <= threshold:
        # nan값을 제외한 값으로만 리스트 생성
        value_list = [i for i in data if not pd.isna(i)]
        
        # 평균값 산출 이후 반올림
        mean = np.mean(value_list)
        mean = round(mean, digits)
               
    return mean

In [2]:
ws_df = pd.read_csv("C:/Users/ITSC/Documents/2015~2024_국내해양데이터_전처리.csv")

In [6]:
# 중복되는 컬럼 제거
ws_df = ws_df.drop(["Unnamed: 1", "지점.1"], axis = 1)

In [7]:
ws_df.head()

,지점,일시,풍속(m/s),풍향(deg),GUST풍속(m/s),현지기압(hPa),습도(%),기온(°C),수온(°C),최대파고(m),유의파고(m),평균파고(m),파주기(sec),파향(deg)
0,21229,2015-01-01 00:00:00,6.3,336.0,11.9,1010.7,96.0,0.2,13.5,3.0,2.3,1.7,7.1,147.0
1,21229,2015-01-01 01:00:00,12.1,302.0,18.1,1012.6,96.0,0.2,13.5,3.4,2.7,1.9,7.1,185.0
2,21229,2015-01-01 02:00:00,10.7,288.0,14.7,1012.1,96.0,-0.2,13.5,3.5,2.7,1.9,6.4,202.0
3,21229,2015-01-01 03:00:00,11.0,295.0,17.8,1011.8,84.0,1.1,13.5,3.9,2.6,1.8,7.1,179.0
4,21229,2015-01-01 04:00:00,14.1,298.0,23.3,1011.6,95.0,0.4,13.5,3.9,3.5,2.5,7.1,189.0


### 데이터가 충분하지 않은 지점 제거

In [60]:
# 지점별 데이터 수량 확인
ws_df["지점"].value_counts()

지점
22103    79498
22105    79260
22108    76963
22106    76554
21229    75906
22104    75547
22183    74904
22102    74660
22101    73749
22107    71270
22184    70080
22187    69530
22188    68417
22190    67878
22189    66444
22185    62970
22186    62312
22192    34198
22298    33010
22191    31594
22297    31348
22300    29496
22299    28473
22301    22780
22302    22434
22303    22390
22304    11813
22305     8474
22309     4335
22310     3984
22311     3967
22193     2322
22194     2321
Name: count, dtype: int64

- 15년 1월 1일부터 24년 5월 19일까지는 3426일, 시간으로는 최대 82224개의 데이터가 존재할 수 있음
- 7만개 이하의 데이터 수량을 가진 지점들을 확인해본 결과 첫 관측 날짜가 15년 1월 1일이 아닌 것이 확인됨
- 15년 1월 1일부터 관측을 시작하지 않은 지점은 데이터에서 제외하기로 결정
    - 결과적으로 11개의 지점만 남기기로 함
    - 동일한 기준으로 중국, 한국 기상데이터도 지점을 제거할 예정

In [80]:
# 데이터 수량이 7만개 이상인 지점 번호만 리스트로 저장
keep_loc = ws_df["지점"].value_counts()[:11]

In [171]:
# keep_loc 에 지점이 포함된 데이터만 남기기
ws_df2 = ws_df[ws_df["지점"].map(lambda x: x in keep_loc)]

In [172]:
# 컬럼 명칭 수정, 괄호 안의 단위 제거
ws_df2.columns = ['지점', '일시', '풍속', '풍향', 'GUST풍속', '현지기압', '습도', '기온', '수온', '최대파고', '유의파고', '평균파고', '파주기', '파향']

In [173]:
# 일시 피처를 datetime 타입으로 변환
ws_df2["일시"] = ws_df2["일시"].map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

C:\Users\ITSC\AppData\Local\Temp\ipykernel_9560\1425953076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ws_df2["일시"] = ws_df2["일시"].map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))


In [174]:
ws_df2 = ws_df2.iloc[:, :-6]

In [175]:
ws_pivot = ws_df2.pivot(index = ["일시"], columns = ["지점"], values = ws_df2.columns[2:])

In [228]:
ws_pivot["습도"].head()

지점,21229,22101,22102,22103,22104,22105,22106,22107,22108,22183,22184
일시,,,,,,,,,,,
2015-01-01 00:00:00,96.0,56.0,80.0,63.0,50.0,52.0,NaN,64.0,69.0,88.0,63.0
2015-01-01 01:00:00,96.0,56.0,96.0,64.0,58.0,55.0,NaN,57.0,90.0,93.0,63.0
2015-01-01 02:00:00,96.0,58.0,87.0,69.0,53.0,55.0,NaN,86.0,89.0,93.0,64.0
2015-01-01 03:00:00,84.0,59.0,73.0,70.0,51.0,59.0,NaN,81.0,87.0,77.0,88.0
2015-01-01 04:00:00,95.0,59.0,64.0,71.0,52.0,65.0,NaN,70.0,86.0,64.0,89.0


### 3시간 단위의 데이터프레임 생성

In [229]:
# 15년 1월 1일부터 24년 5월 19일까지 3시간 단위의 시간 리스트 생성
start_time = ws_df2["일시"].min()
time_list = [start_time + (i * timedelta(hours = 3)) for i in range(0, 3427*8)]

In [178]:
# 시간별로 데이터프레임 생성
time_df = pd.DataFrame(time_list)
time_df.columns = ["일시"]

In [179]:
# time_df가 ws_pivot과 동일한 인덱스와 컬럼을 지니도록 변경
for i in ws_pivot.columns:
    time_df[i] = np.nan
    
time_df = time_df.set_index("일시")
time_df.columns = ws_pivot.columns

In [182]:
# 3시간 단위로 평균값을 구한 후 time_df에 입력. 풍향은 예외
for i in tqdm(time_df.index):
    for j in time_df.drop("풍향", axis = 1).columns:
        # 각각 3개의 시간을 지정
        time0 = i
        time1 = i - timedelta(hours = 1)
        time2 = i - timedelta(hours = 2)
        
        # 3개의 값을 하나의 리스트에 담기
        value_list = []
        
        for time in [time0, time1, time2]:
            try:
                value = ws_pivot.loc[time, j]
                
            except:
                value = np.nan
                
            value_list.append(value)
        
        # nan값이 아닌 데이터로만 이루어진 value_list2
        value_list2 = [i for i in value_list if i != np.nan]
        mean = np.nan
        
        if len(value_list2) != 0:
            mean = np.mean(value_list2)
        
        # 입력
        time_df.loc[i, j] = mean

100%|████████████████████████████████████████████████████████████████████████████| 27408/27408 [15:26<00:00, 29.59it/s]


In [188]:
# 반올림 처리. 습도는 소수점 없이 기존 양식 유지
time_df["풍속"] = time_df["풍속"].apply(lambda x: round(x, 1))
time_df["GUST풍속"] = time_df["GUST풍속"].apply(lambda x: round(x, 1))
time_df["현지기압"] = time_df["현지기압"].apply(lambda x: round(x, 1))
time_df["습도"] = time_df["습도"].apply(lambda x: round(x, 0))
time_df["기온"] = time_df["기온"].apply(lambda x: round(x, 1))

In [192]:
# 15년 1월 1일 데이터 입력
time_df.iloc[0, :] = ws_pivot.iloc[0, :]

In [280]:
time_df.to_csv("./korea_weather_3h.csv")

In [341]:
time_df.index = time_df.index.map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [342]:
time_df.head()

풍속                                                  \
지점                  21229 22101 22102 22103 22104 22105 22106 22107 22108   
일시                                                                          
2015-01-01 00:00:00   6.3  12.8  14.6  14.4   9.9  11.9  12.2  14.0  15.9   
2015-01-01 03:00:00  11.0  12.0  14.0  13.0  10.0  13.0  12.5  14.0  13.0   
2015-01-01 06:00:00  14.0  12.0  13.0  14.0  11.0  13.0  13.1  15.0  14.0   
2015-01-01 09:00:00  12.0  12.0  15.0  14.0  12.0  13.0  12.6  10.0  14.0   
2015-01-01 12:00:00  12.0  10.0  13.0  14.0  12.0  13.0  12.3  14.0  13.0   

                           ...    기온                                      \
지점                  22183  ... 22101 22102 22103 22104 22105 22106 22107   
일시                         ...                                             
2015-01-01 00:00:00   7.5  ...   3.1   1.7   3.2   2.8   2.8   2.0   7.1   
2015-01-01 03:00:00  10.0  ...   3.5   0.4   2.4   1.5   0.6   1.7   6.3   
2015-01-01 06:00:00  11.0  ...   4.4   0.8   1.4  -0.0  -0.9   1.3   5.8   
2015-01-01 09:00:00  11.0  ...   5.0   0.5   1.1  -1.3  -0.8   0.8   5.8   
2015-01-01 12:00:00  10.0  ...   5.0   0.6   1.2  -0.8  -0.7   0.9   5.2   

                                       
지점                  22108 22183 22184  
일시                                     
2015-01-01 00:00:00  -0.5   0.3   5.2  
2015-01-01 03:00:00  -2.1  -0.2   4.5  
2015-01-01 06:00:00  -1.3   0.2   2.2  
2015-01-01 09:00:00  -1.7  -0.8   0.8  
2015-01-01 12:00:00  -2.0   0.4   0.9  

[5 rows x 66 columns]

In [254]:
def horizontal_average(dataframe, datetime, column, threshold = 3, digits = 1):
    '''
    동일한 시간의 여러 지점 데이터를 확인한 후 해당 데이터들의 평균값을 반환하는 함수
    threshold 값을 입력하여 허용하는 NaN 값의 최대치를 설정할 수 있음
    '''  
    # 입력된 일시와 컬럼명에 해당하는 데이터 변수화
    data = dataframe.loc[datetime, column]
    
    # 임시로 nan값 입력
    mean = np.nan
    
    # data에 NaN값의 수가 threshold를 초과하는지 확인
    if data.isna().sum() <= threshold:
        # nan값을 제외한 값으로만 리스트 생성
        value_list = [i for i in data if not pd.isna(i)]
        
        # 평균값 산출 이후 반올림
        mean = np.mean(value_list)
        mean = round(mean, digits)
               
    return mean

In [255]:
horizontal_average(time_df, "2015-01-01 00:00:00", "풍속")

12.2

In [320]:
time_df = pd.read_csv("./korea_weather_3h.csv", header = [0, 1], index_col = 0)

In [328]:
column = "기온"

for time in tqdm(time_df.index):
    mean = horizontal_average(time_df, time, column)
    time_df.loc[time, column] = time_df.loc[time, column].fillna(mean).tolist()

100%|███████████████████████████████████████████████████████████████████████████| 27416/27416 [00:46<00:00, 594.45it/s]


In [330]:
column = "풍속"

for time in tqdm(time_df.index):
    mean = horizontal_average(time_df, time, column)
    time_df.loc[time, column] = time_df.loc[time, column].fillna(mean).tolist()

100%|███████████████████████████████████████████████████████████████████████████| 27416/27416 [00:47<00:00, 580.68it/s]


In [332]:
column = "현지기압"

for time in tqdm(time_df.index):
    mean = horizontal_average(time_df, time, column)
    time_df.loc[time, column] = time_df.loc[time, column].fillna(mean).tolist()

100%|███████████████████████████████████████████████████████████████████████████| 27416/27416 [00:48<00:00, 565.31it/s]


In [338]:
column = "습도"

for time in tqdm(time_df.index):
    mean = horizontal_average(time_df, time, column, digits = 0)
    time_df.loc[time, column] = time_df.loc[time, column].fillna(mean).tolist()

100%|███████████████████████████████████████████████████████████████████████████| 27416/27416 [00:45<00:00, 608.13it/s]


In [334]:
column = "GUST풍속"

for time in tqdm(time_df.index):
    mean = horizontal_average(time_df, time, column)
    time_df.loc[time, column] = time_df.loc[time, column].fillna(mean).tolist()

100%|███████████████████████████████████████████████████████████████████████████| 27416/27416 [00:46<00:00, 583.68it/s]


In [335]:
time_df.head()

풍속                                                  \
지점                  21229 22101 22102 22103 22104 22105 22106 22107 22108   
일시                                                                          
2015-01-01 00:00:00   6.3  12.8  14.6  14.4   9.9  11.9  12.2  14.0  15.9   
2015-01-01 03:00:00  11.0  12.0  14.0  13.0  10.0  13.0  12.5  14.0  13.0   
2015-01-01 06:00:00  14.0  12.0  13.0  14.0  11.0  13.0  13.1  15.0  14.0   
2015-01-01 09:00:00  12.0  12.0  15.0  14.0  12.0  13.0  12.6  10.0  14.0   
2015-01-01 12:00:00  12.0  10.0  13.0  14.0  12.0  13.0  12.3  14.0  13.0   

                           ...    기온                                      \
지점                  22183  ... 22101 22102 22103 22104 22105 22106 22107   
일시                         ...                                             
2015-01-01 00:00:00   7.5  ...   3.1   1.7   3.2   2.8   2.8   2.0   7.1   
2015-01-01 03:00:00  10.0  ...   3.5   0.4   2.4   1.5   0.6   1.7   6.3   
2015-01-01 06:00:00  11.0  ...   4.4   0.8   1.4  -0.0  -0.9   1.3   5.8   
2015-01-01 09:00:00  11.0  ...   5.0   0.5   1.1  -1.3  -0.8   0.8   5.8   
2015-01-01 12:00:00  10.0  ...   5.0   0.6   1.2  -0.8  -0.7   0.9   5.2   

                                       
지점                  22108 22183 22184  
일시                                     
2015-01-01 00:00:00  -0.5   0.3   5.2  
2015-01-01 03:00:00  -2.1  -0.2   4.5  
2015-01-01 06:00:00  -1.3   0.2   2.2  
2015-01-01 09:00:00  -1.7  -0.8   0.8  
2015-01-01 12:00:00  -2.0   0.4   0.9  

[5 rows x 66 columns]

In [362]:
# 3시간 단위로 평균값을 구한 후 time_df에 입력. 풍향만
for i in tqdm(time_df.index):
    for j in time_df["풍향"].columns:
        # 각각 3개의 시간을 지정
        time0 = i
        time1 = i - timedelta(hours = 1)
        time2 = i - timedelta(hours = 2)
        
        # 3개의 값을 하나의 리스트에 담기
        value_list = []
        
        for time in [time0, time1, time2]:
            try:
                value = ws_pivot["풍향"].loc[time, j]
                
            except:
                value = np.nan
                
            value_list.append(value)
        
        # nan값이 아닌 데이터로만 이루어진 value_list2
        value_list2 = [i for i in value_list if i != np.nan]
        mean = np.nan
        
        if len(value_list2) != 0:
            mean = average_angle_ignore_nan(value_list2)
        
        # 입력
        time_df.loc[i, ("풍향", j)] = mean

100%|████████████████████████████████████████████████████████████████████████████| 27416/27416 [05:32<00:00, 82.35it/s]


In [355]:
ws_pivot.loc["2015-01-01 00:00:00", ("풍향", 21229)]

336.0

In [363]:
time_df["풍향"]

지점,21229,22101,22102,22103,22104,22105,22106,22107,22108,22183,22184
일시,,,,,,,,,,,
2015-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-19 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-19 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-19 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for time in tqdm(time_df.index):
    data = time_df.loc[time, "풍향"]
    mean = average_angle_ignore_nan(data)